In [ ]:
import logging

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

In [ ]:
from pyinstrument import Profiler


profiler = Profiler()

with profiler:
    pass

profiler.write_html('../temp/features.html')

In [ ]:
from labda_parsers import Sens
from acti_motus import Features

features = Features(chunks=False)
reader = Sens()

files = {
    'thigh': '../test_data/thigh.bin',
    'trunk': '../test_data/trunk.bin',
    'calf': '../test_data/calf.bin',
    'arm': '../test_data/arm.bin',
}

for name, file in files.items():
    df = reader.from_file(file)

    # df['acc_x'] = -df['acc_x'] # Setting incorrect sensor placement
    # df.iloc[20000:200000] = 0 # Imputting non-wear
    # df.iloc[0:250000] = 0 # Imputting non-wear

    f = features.extract(df)
    f.to_parquet(f'../temp/{name}_features.parquet')

In [ ]:
import pandas as pd
from acti_motus import Thigh, References
from pyinstrument import Profiler


file = pd.read_parquet('../temp/thigh_features.parquet')

profiler = Profiler()

with profiler:
    references = References()
    activities = Thigh('Sens').detect_activities(file, references)

profiler.write_html('../temp/activities.html')

# activities.to_parquet('../temp/thigh_activities.parquet')

In [ ]:
from acti_motus import Trunk
import pandas as pd

activities = pd.read_parquet('../temp/thigh_activities.parquet')
features = pd.read_parquet('../temp/trunk_features.parquet')

references = None
df = Trunk().detect_activities(features, activities, references)
df.to_parquet('../temp/trunk_activities.parquet')

In [ ]:
import logging
from acti_motus import Calf
import pandas as pd

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

activities = pd.read_parquet('../temp/trunk_activities.parquet')
features = pd.read_parquet('../temp/calf_features.parquet')

df = Calf().detect_activities(features, activities)
df.to_parquet('../temp/calf_activities.parquet')
df['activity'].value_counts()

In [ ]:
from acti_motus import Arm
import pandas as pd

features = pd.read_parquet('../temp/arm_features.parquet')
df = Arm().detect_activities(features)

df.value_counts(dropna=False)

In [ ]:
from acti_motus.classifications.references import References

references = {
    'thigh': {
        'value': -0.201,
        'expires': '2024-09-03 12:05:51+00:00',
    },
    'trunk': {
        'value': [1.320, 0.247, -0.038],
        'expires': '2024-09-05 05:05:51+00:00',
    },
    'calibrations': [
        {
            'start': '2024-09-03 08:08:51+00:00',
            'end': '2024-09-03 08:09:11+00:00',
            'ttl': '24h',
        },
        {
            'start': '2024-09-02 10:36:31+00:00',
            'end': '2024-09-02 20:36:51+00:00',
            'ttl': '12h',
        },
        {
            'start': '2024-09-03 15:16:23+00:00',
            'end': '2024-09-03 17:17:03+00:00',
            'ttl': '96h',
        },
    ],
}

In [ ]:
from acti_motus import Activities, References, Exposures
import pandas as pd

from pyinstrument import Profiler

thigh = pd.read_parquet('../temp/thigh_features.parquet')
trunk = pd.read_parquet('../temp/trunk_features.parquet')
# calf = pd.read_parquet('../temp/calf_features.parquet')
# arm = pd.read_parquet('../temp/arm_features.parquet')

profiler = Profiler()

with profiler:
    activities, references = Activities(vendor='Sens', chunks=True).detect(
        thigh=thigh, trunk=trunk, references=references
    )
    activities.to_parquet('../temp/activities.parquet')
    exposures = Exposures().generate(activities)

profiler.write_html('../temp/activities.html')

In [ ]:
references

In [ ]:
from acti_motus import Activities, References, Exposures
import pandas as pd

df = pd.read_parquet('../temp/activities.parquet')
Exposures().generate(df)